# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [5]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [24]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [54]:
data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
...,...,...,...,...,...,...,...
68579,94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24
68580,57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15
68581,36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10
68582,43270,Devon,Mc Daniel,4328852,211,Wine - Prosecco Valdobiaddene,11


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [312]:
data_grouped=data.groupby(["CustomerID","ProductName"]).sum()

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [313]:
pivot_table=pd.pivot_table(data_grouped,fill_value=0,columns=["CustomerID"],index="ProductName",values="Quantity")


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [314]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot_table.T, 'euclidean'))), 
                          index=pivot_table.columns,columns=pivot_table.columns)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [315]:
id_costumer=1034

most_similar_clients=distances[id_costumer].sort_values(ascending=False).index.values[1:6]



## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [316]:
similar_client_data=data_grouped.loc[most_similar_clients]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [317]:
similar_client_data=similar_client_data.groupby("ProductName").sum().sort_values(by="Quantity",ascending=False)

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [347]:
s=similar_client_data.merge(data_grouped.loc[id_costumer],on="ProductID",how="right")

s=s[s.Quantity_x.isna()].sort_values(by="Quantity_y",ascending=False)["ProductID"][0:5]

data.loc[s]["ProductName"].values

array(['Beer - Original Organic Lager', 'Rosemary - Primerba, Paste',
       'Arizona - Green Tea', 'Cheese Cloth No 100',
       'Arizona - Green Tea'], dtype=object)

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [348]:
recomendations={}
ID_costumers=set(data["CustomerID"].values)

for costumer in ID_costumers:
    most_similar_clients=distances[costumer].sort_values(ascending=False).index.values[1:6]
    similar_client_data=data_grouped.loc[most_similar_clients]
    similar_client_data=similar_client_data.groupby("ProductName").sum().sort_values(by="Quantity",ascending=False)
    s=similar_client_data.merge(data_grouped.loc[id_costumer],on="ProductID",how="right")
    s=s[s.Quantity_x.isna()].sort_values(by="Quantity_y",ascending=False)["ProductID"][0:5]
    recomendations.update({costumer:data.loc[s]["ProductName"].values})

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [386]:
df=pd.DataFrame.from_dict(recomendations,orient="index").reset_index()
df.columns=["CustomerID","Product 1","Product 2","Product 3","Product 4","Product 5"]
df

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,83973,Arizona - Green Tea,"Rosemary - Primerba, Paste",Arizona - Green Tea,Cheese Cloth No 100,Beer - Original Organic Lager
1,59399,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Cheese Cloth No 100,Arizona - Green Tea,Arizona - Green Tea
2,92168,Arizona - Green Tea,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Cheese Cloth No 100,Arizona - Green Tea
3,49159,Arizona - Green Tea,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Bread - Hot Dog Buns,Cheese Cloth No 100
4,18441,Arizona - Green Tea,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Arizona - Green Tea,Cheese Cloth No 100
5,22536,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Bread - Hot Dog Buns,Arizona - Green Tea,Cheese Cloth No 100
6,86028,"Rosemary - Primerba, Paste",Beer - Original Organic Lager,Arizona - Green Tea,Arizona - Green Tea,Arizona - Green Tea
7,75791,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Cheese Cloth No 100,Bread - Hot Dog Buns,Arizona - Green Tea
8,96272,Bread - Hot Dog Buns,Arizona - Green Tea,Arizona - Green Tea,Arizona - Green Tea,Beer - Original Organic Lager
9,32785,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Arizona - Green Tea,Cheese Cloth No 100,Bread - Hot Dog Buns


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [387]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot_table.T, 'minkowski'))), 
                          index=pivot_table.columns,columns=pivot_table.columns)

In [388]:
recomendations={}
ID_costumers=set(data["CustomerID"].values)

for costumer in ID_costumers:
    most_similar_clients=distances[costumer].sort_values(ascending=False).index.values[1:6]
    similar_client_data=data_grouped.loc[most_similar_clients]
    similar_client_data=similar_client_data.groupby("ProductName").sum().sort_values(by="Quantity",ascending=False)
    s=similar_client_data.merge(data_grouped.loc[id_costumer],on="ProductID",how="right")
    s=s[s.Quantity_x.isna()].sort_values(by="Quantity_y",ascending=False)["ProductID"][0:5]
    recomendations.update({costumer:data.loc[s]["ProductName"].values})

In [389]:
df=pd.DataFrame.from_dict(recomendations,orient="index").reset_index()
df.columns=["CustomerID","Product 1","Product 2","Product 3","Product 4","Product 5"]

In [390]:
df

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,83973,Arizona - Green Tea,Beer - Original Organic Lager,Cheese Cloth No 100,Bread - Hot Dog Buns,Beer - Original Organic Lager
1,59399,"Rosemary - Primerba, Paste",Bread - Hot Dog Buns,Cheese Cloth No 100,Arizona - Green Tea,Arizona - Green Tea
2,92168,Arizona - Green Tea,Arizona - Green Tea,"Rosemary - Primerba, Paste",Cheese Cloth No 100,Arizona - Green Tea
3,49159,Arizona - Green Tea,Bread - Hot Dog Buns,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Bread - Hot Dog Buns
4,18441,Arizona - Green Tea,Beer - Original Organic Lager,Cheese Cloth No 100,Arizona - Green Tea,Bread - Hot Dog Buns
5,22536,Arizona - Green Tea,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Cheese Cloth No 100,Bread - Hot Dog Buns
6,86028,Arizona - Green Tea,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Arizona - Green Tea,Bread - Hot Dog Buns
7,75791,Arizona - Green Tea,Arizona - Green Tea,Bread - Hot Dog Buns,Beer - Original Organic Lager,Cheese Cloth No 100
8,96272,Beer - Original Organic Lager,Arizona - Green Tea,Bread - Hot Dog Buns,Cheese Cloth No 100,Arizona - Green Tea
9,32785,Arizona - Green Tea,Beer - Original Organic Lager,"Rosemary - Primerba, Paste",Arizona - Green Tea,Bread - Hot Dog Buns
